In [1]:
import torch
from torch import nn
from config import IMAGE_TRANSFORM, VIDEO_TRANSFORM
from torch.utils.data import Dataset, DataLoader
import PIL
from PIL import Image
import os
from preprocessing import uniform_crop, Spacial_Crop, get_clip_timepoints
import matplotlib.pyplot as plt
import clip
from typing import Tuple

from pytorchvideo.data.clip_sampling import ConstantClipsPerVideoSampler
from pytorchvideo import transforms as pv_transforms
from pytorchvideo.data.encoded_video import EncodedVideo
import av

import pexelsapi
from pexelsapi.pexels import Pexels
import dotenv
import requests
import cv2
import numpy as np
from tqdm import tqdm

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Pl

In [2]:
def load_and_transform_text(texts, device="cpu"):
    if texts is None: return None
    tokens = [clip.tokenize(text).unsqueeze(0).to(device) for text in texts]
    tokens = torch.cat(tokens, dim=0)
    tokens = tokens.squeeze(dim = 1)
    return tokens

In [3]:
class ImageText_DataLoader(Dataset):
    def __init__(self, image_paths, transform, device = "cpu"):
        super().__init__()
        
        if image_paths is None: return None
        
        self.image_outputs = []
        self.text_outputs = []

        for image_path in os.listdir(image_paths):
            if image_path.endswith(".jpg") or image_path.endswith(".jpeg") or image_path.endswith(".png"):
                with open(os.path.join(image_paths, image_path), "rb") as im:
                    image = Image.open(im).convert("RGB")
                    image = transform(img = image)
                    self.image_outputs.append(image)
                    self.text_outputs.append(image_path.split('.')[0])
        self.text_outputs = load_and_transform_text(self.text_outputs)
        # return torch.stack(image_outputs, dim=0)

    def __len__(self):
        return len(self.image_outputs) or len(self.text_outputs)
    
    def __getitem__(self, index):
        return (self.image_outputs[index], self.text_outputs[index])

ds = ImageText_DataLoader(image_paths = "Data/Image_Data", transform = IMAGE_TRANSFORM)
dl = DataLoader(ds, batch_size=5)
next(iter(dl))[1].shape

torch.Size([5, 77])

In [94]:
class VideoText_DataLoader(Dataset):
    def __init__(self, video_paths: str, transform, clip_duration=2, clips_per_video=5, device = "cpu"):
        super().__init__()

        self.video_outputs = []
        self.text_outputs = []
        
        video_transform = transform

        clip_sampler = ConstantClipsPerVideoSampler(
            clip_duration=clip_duration, clips_per_video=clips_per_video, 
        )

        frame_sampler = pv_transforms.UniformTemporalSubsample(num_samples=2)
        for video_path in os.listdir(video_paths): 
            if video_path.endswith(".mp4"):
                encoded_video = EncodedVideo.from_path(
                    file_path=f"Data/Video_Data/{video_path}",
                    decode_audio=False,
                    decoder="pyav"
                )

                all_frames = []

                clip_time_points = get_clip_timepoints(clip_sampler=clip_sampler, duration=encoded_video.duration)

                for clip_time_point in clip_time_points:
                    clip = encoded_video.get_clip(clip_time_point[0], clip_time_point[1])
                    if clip is None: ValueError("No Clip Found") 
                    frames = frame_sampler(clip["video"]) / 255.0

                    all_frames.append(frames)
                all_videos = [video_transform(frame) for frame in all_frames]
                self.video_outputs.append(Spacial_Crop(crop_size = 224, num_crops = 3)(all_videos))
                self.text_outputs.append(video_path.split('.')[0])
        self.text_outputs = load_and_transform_text(self.text_outputs)

    def __len__(self):
        return len(self.video_outputs)
    
    def __getitem__(self, index):
        return self.video_outputs[index], self.text_outputs[index]

ds = VideoText_DataLoader(video_paths = "Data/Video_Data", transform = VIDEO_TRANSFORM)
dl = DataLoader(ds, batch_size=5)
next(iter(dl))[0].shape

torch.Size([5, 15, 3, 2, 224, 224])

In [95]:
next(iter(dl))[1].shape

torch.Size([5, 77])

In [96]:
PEXEL = dotenv.dotenv_values()['PEXEL']

pexel = Pexels(PEXEL)
search_videos = pexel.search_videos(query='ocean', orientation='', size='', color='', locale='', page=1, per_page=15)
search_videos

{'page': 1,
 'per_page': 15,
 'videos': [{'id': 1918465,
   'width': 3840,
   'height': 2160,
   'duration': 15,
   'full_res': None,
   'tags': [],
   'url': 'https://www.pexels.com/video/bird-s-eye-view-of-ocean-waves-1918465/',
   'image': 'https://images.pexels.com/videos/1918465/free-video-1918465.jpg?auto=compress&cs=tinysrgb&fit=crop&h=630&w=1200',
   'avg_color': None,
   'user': {'id': 574687,
    'name': 'Ruvim Miksanskiy',
    'url': 'https://www.pexels.com/@digitech'},
   'video_files': [{'id': 9228888,
     'quality': 'uhd',
     'file_type': 'video/mp4',
     'width': 3840,
     'height': 2160,
     'fps': 23.979999542236328,
     'link': 'https://videos.pexels.com/video-files/1918465/1918465-uhd_3840_2160_24fps.mp4',
     'size': 48292630},
    {'id': 9228993,
     'quality': 'sd',
     'file_type': 'video/mp4',
     'width': 960,
     'height': 540,
     'fps': 23.979999542236328,
     'link': 'https://videos.pexels.com/video-files/1918465/1918465-sd_960_540_24fps.mp4',

In [89]:
def download_direct_video(video_url, output_path="downloaded_video.mp4"):
    response = requests.get(video_url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(output_path, 'wb') as file, tqdm(
        desc="Downloading video",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    return output_path

def resize_with_pad(image: np.array, 
                    new_shape: Tuple[int, int], 
                    padding_color: Tuple[int] = (0, 0, 0)) -> np.array:
    original_shape = (image.shape[1], image.shape[0])
    ratio = float(max(new_shape))/max(original_shape)
    new_size = tuple([int(x*ratio) for x in original_shape])
    image = cv2.resize(image, new_size)
    delta_w = new_shape[0] - new_size[0]
    delta_h = new_shape[1] - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
    return image

def pad_to_square(frame):
    h, w, _ = frame.shape
    size = max(h, w)
    top = (size - h) // 2
    bottom = size - h - top
    left = (size - w) // 2
    right = size - w - left
    padded = cv2.copyMakeBorder(frame, top, bottom, left, right,
                                 cv2.BORDER_CONSTANT, value=[0, 0, 0])  # black padding
    return padded

def process_and_save_video(input_path, output_path_video, output_path_image, name, new_shape=(256, 256), fps=30):
    cap = cv2.VideoCapture(input_path)
    output_path_video = output_path_video + name + '.mp4'

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' or 'avc1' for AVI/MOV

    out = cv2.VideoWriter(output_path_video, fourcc, fps, new_shape)
    
    frame_count = 0
    frame_flag = False
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed = resize_with_pad(frame, new_shape)
        out.write(processed)
        if not frame_flag: plt.imsave(output_path_image + name + '.jpg', cv2.resize(pad_to_square(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)), (256, 256)))
        frame_flag = True
        if frame_count == 180: break
        frame_count += 1

    cap.release()
    out.release()
    print(f"Saved processed video to {output_path_video}")
    os.remove(input_path)

queries = ["cat", "dog", "ocean", "city", "sunset", "baseball"]

for query in queries:
    search_videos = pexel.search_videos(query=query, orientation='', size='', color='', locale='', page=1, per_page=5)

    for idx in range(len(search_videos['videos'])):
        video_info = search_videos['videos'][idx]
        video_url = video_info['video_files'][-1]['link']
        video_path = download_direct_video(video_url)

        url = video_info['url']
        name = ' '.join(url.split('/')[-2].split('-')[:-1])

        process_and_save_video(video_path, output_path_video = f"Data/Video_Data/", output_path_image = "Data/Image_Data/", name = name)

Saved processed video to Data/Video_Data/video of funny cat.mp4


Saved processed video to Data/Video_Data/video of a tabby cat.mp4


Saved processed video to Data/Video_Data/little kitten playing his toy mouse.mp4


Saved processed video to Data/Video_Data/cute cat.mp4


Saved processed video to Data/Video_Data/the full facial features of a pet cat.mp4


...

Saved processed video to Data/Video_Data/a baseball game in a stadium.mp4


Saved processed video to Data/Video_Data/close up view of person playing with baseball.mp4


Saved processed video to Data/Video_Data/aerial view of a baseball field.mp4


Saved processed video to Data/Video_Data/baseball player hitting the baseball.mp4


Saved processed video to Data/Video_Data/aerial view of a baseball field.mp4
